In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *


In [0]:
df=spark.read.format('csv').option('inferSchema',True).option('header',True).load('/Volumes/workspace/pyspark/bigmart/BigMart Sales.csv')

In [0]:
df.display()

## Select

In [0]:
df_sel=df.select('Item_Identifier','Item_Weight','Item_Fat_Content')
df_sel.display()

In [0]:
df.select(col("Item_Fat_Content"),col('Item_Weight'),col('Item_Identifier')).display()

## Alias

In [0]:
df.select(col("Item_Identifier").alias('Item ID')).display()

## Filter/Where

In [0]:
df.filter(col('Item_Fat_Content')=='Regular').display()

In [0]:
#item weight >5 and item type soft drinks
df.filter((col('Item_Weight')<10) & (col('Item_Type')=='Soft Drinks')).display()


In [0]:
df.filter((col('Outlet_Location_Type').isin('Tier 1','Tier 2') )& (col('Outlet_Size').isNull())).display()

## Withcolumn Renamed

In [0]:
df.withColumnRenamed('Item_Weight','Item wgt').display()

In [0]:
df.select(col('Item_Fat_Content')).distinct().display()

## With Column ->New col /Modify column

In [0]:
df=df.withColumn("Category",when(col('Item_Fat_Content')=='Low Fat','Good').when(col('Item_Fat_Content')=='Regular','OkOk').otherwise('shortform')).withColumn("Multiplied value",col('Item_Weight')*col('Item_MRP'))
              

In [0]:
df.display()

In [0]:
df=df.withColumn('Item_Fat_Content',when(col('Item_Fat_Content')=='Low Fat','LF').when(col('Item_Fat_Content')=='Regular','Reg')) #trying to modify existing column

In [0]:
df.display()

## Type Casting

In [0]:
df=df.withColumn('Item_Weight',col('Item_Weight').cast(StringType()))
df.printSchema()

## Sort/Order By

In [0]:
df.sort(col('Item_MRP').desc()).display()

In [0]:
#sorting with two columns
df.sort(['Item_Weight','Item_MRP'],ascending=[0,1]).display() # 0 means false->descending 1->true so ascending

## Limit

In [0]:
df.limit(10).display()

## Drop and drop duplicates

In [0]:
df.drop('Item_Identifier').display() 

In [0]:
df.dropDuplicates()

In [0]:
df.drop_duplicates(subset=['Item_Type']).display()

## Union

In [0]:
schema=' ID INT , Name STRING'
Data=[(1,'Anish'),(2,'abc')]
schema2=' ID INT , Name STRING'
Data2=[(1,'Anish'),(2,'abc')]
df_1=spark.createDataFrame(data=Data,schema=schema)
df_2=spark.createDataFrame(data=Data2,schema=schema2)

df_newone=df_1.unionAll(df_2)
df_newone.display()

In [0]:
schema=' Name STRING , ID STRING'
Data=[('Anish',1),('abc',2)]
schema2=' ID STRING , Name STRING'
Data2=[(1,'Anish'),(2,'abc')]
df_1=spark.createDataFrame(data=Data,schema=schema)
df_2=spark.createDataFrame(data=Data2,schema=schema2)
df_newone=df_1.unionByName(df_2)
df_newone.display()

## String Functions

In [0]:
df.select(lower(col('Item_Fat_Content')).alias('Fat Content')).distinct().display()

## Date Functions

In [0]:
df=df.withColumn('Current_date',current_date()).withColumn('futuredate',date_add(col('Current_date'),7)).withColumn('weekbefore',date_sub(col('Current_date'),7))
df.display()

In [0]:
#DATEDIFF
df.withColumn('datediff',date_diff('Current_date','futuredate')).display()

## DATE Format

In [0]:
df=df.withColumn('Current date',date_format('Current_date','dd-MM-yyyy'))
df.display()

## Handling Nulls

In [0]:
df.dropna('any') #drops any column row that has null
df.dropna('all') # drops rows where all values are null

In [0]:
df=df.dropna(subset=['Item_Weight'])
df.display()

## Filling NA with Values

In [0]:
df=df.withColumn('Outlet_Size',when(col('Outlet_Size').isNull(),'Unkown').otherwise(col('Outlet_Size')))
df.display()

In [0]:
df=df.fillna('unkown',subset=['Item_Fat_Content'])
df.display()